In [58]:
import ee
ee.Initialize()
import matplotlib.pyplot as plt
from IPython.display import Image
from datetime import datetime, timedelta
import folium

LOAD GEE VECTOR ASSETS

In [99]:
bnd = ee.FeatureCollection("users/wfphqgis/wfp_bnd_inform2019")
global_dams = ee.FeatureCollection("users/wfphqgis/GRanD_dams_v1_1")
dfo_flood = ee.FeatureCollection("users/wfphqgis/DFO_HistoricalFloodEvents")
country = ee.FeatureCollection("users/wfphqgis/wld_wfp_acrcountries_wfp_2018")
aoi = ee.FeatureCollection(bnd).filter(ee.Filter.eq('iso3', 'PHL'))
print(aoi)

ee.FeatureCollection({
  "type": "Invocation", 
  "arguments": {
    "filter": {
      "type": "Invocation", 
      "arguments": {
        "leftField": "iso3", 
        "rightValue": "PHL"
      }, 
      "functionName": "Filter.equals"
    }, 
    "collection": {
      "type": "Invocation", 
      "arguments": {
        "tableId": "users/wfphqgis/wfp_bnd_inform2019"
      }, 
      "functionName": "Collection.loadTable"
    }
  }, 
  "functionName": "Collection.filter"
})


SET DATA PARAMETERS

In [100]:
day0 = datetime.today()
hrs24 = day0 - timedelta(days=1)
hrs48 = day0 - timedelta(days=2)
hrs72 = day0 - timedelta(days=3)
day7 = day0 - timedelta(days=7)
print day0
print hrs24
print hrs72
print day7

2018-12-24 12:41:39.578335
2018-12-23 12:41:39.578335
2018-12-21 12:41:39.578335
2018-12-17 12:41:39.578335


LOAD GSMaP RASTER DATA (OPERATIONAL)

In [172]:
dataset = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational').filter(ee.Filter.date(hrs24, day0))
precipitation = dataset.select('hourlyPrecipRateGC')

gpm_precip_24h = precipitation.filter(ee.Filter.date(hrs24, day0))
gpm_precip_48h = precipitation.filter(ee.Filter.date(hrs48, day0))
gpm_precip_72h = precipitation.filter(ee.Filter.date(hrs72, day0))
gpm_precip_day7 = precipitation.filter(ee.Filter.date(day7, day0))

gpm_stat_24h = gpm_precip_24h.sum().clip(aoi)
gpm_stat_48h = gpm_precip_48h.sum().clip(aoi)
gpm_stat_72h = gpm_precip_72h.sum().clip(aoi)
gpm_stat_day7 = gpm_precip_day7.sum().clip(aoi)

#url = gpm_stat_day7.getThumbUrl()
#Image(url=url)

CHARTING (REDUCER) SERVER IS DOWN! Try Agrain Later

In [174]:
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.sum(), aoi)
    return image.set(dict)

withSum = gpm_precip_72h.map(setProperty)

print withSum.aggregate_array('hourlyPrecipRateGC').getInfo()

[2094.919703803099]


In [119]:
#image = ee.Image('USGS/SRTMGL1_003')
dataset = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
url = image.getThumbUrl()
Image(url=url)

In [24]:
s1_aoi = ee.Geometry.Point(45.7521 , 32.6024).buffer(150000)
collections =  ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(s1_aoi).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).select('VV')

collection = collections.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

pre = collection.filterDate('2018-11-10', '2018-11-19').mosaic().clip(s1_aoi)
post = collection.filterDate('2018-11-22', '2018-11-28').mosaic().clip(s1_aoi)

SMOOTHING_RADIUS = 1000
DIFF_UPPER_THRESHOLD = -1.5

diff_smoothed = post_c.focal_median(SMOOTHING_RADIUS, 'circle', 'meters').subtract(pre_c.focal_median(SMOOTHING_RADIUS, 'circle', 'meters'));

srtm = ee.Image('USGS/SRTMGL1_003').clip(s1_aoi)
slope = ee.Terrain.slope(srtm)
slope_f = slope.lte(8)
aspect = ee.Terrain.aspect(srtm)
diff_thresholded = diff_smoothed.lt(DIFF_UPPER_THRESHOLD)
inundated = diff_thresholded.multiply(slope_f)

pre_c = pre.clip(s1_aoi)
post_c = post.clip(s1_aoi)
url = inundated.getThumbUrl()
Image(url=url)

In [26]:
dataset = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
occurrence = dataset.select('occurrence').clip(s1_aoi)
url = occurrence.getThumbUrl()
Image(url=url)

In [27]:
pop = ee.ImageCollection('WorldPop/POP').filter(ee.Filter.equals('year', 2015)).filter(ee.Filter.equals('UNadj', 'yes')).mosaic().clip(s1_aoi)
url = pop.getThumbUrl()
Image(url=url)

In [ ]:
'''
// RAINFALL
var from = ee.Date('2018-11-18');
var to = ee.Date('2018-11-28');

//var from = ee.Date('2018-11-18');
//var to = ee.Date('2018-11-27');
//var now = Date.now();

var dataset = ee.Image('JRC/GSW1_0/GlobalSurfaceWater'); //band: 'occurence'

var s1_aoi = ee.Geometry.Point(45.7521 , 32.6024).buffer(150000); // Wasit Governate
//var s1_aoi = ee.Geometry.Point(43.2358582 , 35.4795323).buffer(150000); // Beledweyne
//var s1_aoi = geometry.buffer(150000)

Map.centerObject(s1_aoi,8);

var collections =  ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(s1_aoi)
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
.select('VV');

var collection = collections.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));

var pre = collection.filterDate('2018-11-10', '2018-11-19').mosaic().clip(s1_aoi); //add s to mosaic
var post = collection.filterDate('2018-11-22', '2018-11-28').mosaic().clip(s1_aoi); // add s to mosaic
print(post)

var pre_c = pre.clip(s1_aoi);
var post_c = post.clip(s1_aoi);

var SMOOTHING_RADIUS = 1000;//Spatial Filter (test @ 1km)
var DIFF_UPPER_THRESHOLD = -1.5;

var diff_smoothed = post_c.focal_median(SMOOTHING_RADIUS, 'circle', 'meters')
    .subtract(pre_c.focal_median(SMOOTHING_RADIUS, 'circle', 'meters'));

var srtm = ee.Image('USGS/SRTMGL1_003').clip(s1_aoi);
var slope = ee.Terrain.slope(srtm);
//Map.addLayer(srtm, {min:0, max: 1000, palette:['black', 'white']})
var slope_f = slope.lte(8); //% slope
//var aspect = ee.Terrain.aspect(srtm);
//var hillshade = aspect.divide(360).multiply(Math.PI).sin();
var diff_thresholded = diff_smoothed.lt(DIFF_UPPER_THRESHOLD);
//var inundated = diff_thresholded.multiply(slope_f);
//Map.addLayer(pre_c, {min:-30,max:0}, 'Before flood');
//Map.addLayer(post_c, {min:-30,max:0}, 'After flood');
//Map.addLayer(post_c.subtract(pre_c), {min:-10,max:10}, 'After - before', 0); 

//####################################################
var dataset = ee.Image('JRC/GSW1_0/GlobalSurfaceWater'); //band: 'occurence'
var occurrence = dataset.select('occurrence').clip(s1_aoi);
//occurrence / %	0	100 / The frequency with which water was present.
//change_abs /%	-100	100	/ Absolute change in occurrence between two epochs: 1984-1999 vs 2000-2015.
//change_norm / %	-100	100	/Normalized change in occurrence. (epoch1-epoch2)/(epoch1+epoch2) * 100
// seasonality / 0	12 / Number of months water is present.
// recurrence / %	0	100/ The frequency with which water returns from year to year.

var monthlyRecurrence = dataset.select('monthly_recurrence');

var occurrenceVis = {
  min: 0.0,
  max: 100.0,
  palette: ['ffffff', 'ffbbbb', '0000ff'],
};

var VIS_WATER_MASK = {
  palette: ['white', 'black']
};

// Create Water Mask
var water_mask = occurrence.lte(20).unmask(20).clip(s1_aoi);
var s2water_mask = occurrence.gte(20).unmask(20).clip(s1_aoi);


// Population/Settlements
var pop = ee.ImageCollection('WorldPop/POP')
  .filter(ee.Filter.equals('year', 2015))
  .filter(ee.Filter.equals('UNadj', 'yes'))
  .mosaic().clip(s1_aoi);

//print(pop)

// GHS
var ghs_bu = ee.Image("JRC/GHSL/P2016/BUILT_LDSMT_GLOBE_V1").clip(s1_aoi); //built-up
//var ghs_pop = ee.ImageCollection("JRC/GHSL/P2016/POP_GPW_GLOBE_V1").clip(AOI); //pop grid
//var ghs_sett = ee.ImageCollection("JRC/GHSL/P2016/SMOD_POP_GLOBE_V1").clip(AOI); //settlements

// GPM
var gpm_band = 'precipitationCal';
var gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V05')
          .filterDate(from, to)
          .select(gpm_band); //Add to.Bands so we can query
          
/*
print(ui.Chart.image.series(gpm,s1_aoi, ee.Reducer.mean())
    .setChartType('BarChart')
    .setOptions({
      title: 'Daily Precipitaion (in mm) based on: ' + gpm_band,
      hAxis: {title: 'GPM Observation Dates'},
      vAxis: {title: 'Observed Rainfall (mm)'}}));
*/    

var dataset = ee.Image('CGIAR/SRTM90_V4');
var elevation = dataset.select('elevation');
var lowland = elevation.lte(30).clip(s1_aoi)
var slope = ee.Terrain.slope(elevation);

var water_masked = diff_thresholded.updateMask(water_mask)
var slope_masked = water_masked.updateMask(slope_f)


// Map the function over one year of data and take the median.
// Load Sentinel-2 TOA reflectance data.
var s2collection = ee.ImageCollection('COPERNICUS/S2')
    .filterDate(from, to);
    // Pre-filter to get less cloudy granules.
    //.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90));

var s2bands = ['B8', 'B4', 'B3', 'QA60']

var select = s2collection.select(s2bands).mosaic().clip(s1_aoi);
var s2_masked = select.updateMask(s2water_mask)

function maskL8sr(image) {
  // Bits 3 and 5 are cloud shadow and cloud, respectively.
  var cloudShadowBitMask = (1 << 3);
  var cloudsBitMask = (1 << 5);
  // Get the pixel QA band.
  var qa = image.select('pixel_qa');
  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
                 .and(qa.bitwiseAnd(cloudsBitMask).eq(0));
  return image.updateMask(mask);
}

var dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                  .filterDate(from, to)
                  .map(maskL8sr);

var l8bands = ['B4', 'B3', 'B2']

var l8 = dataset.select(l8bands).mosaic().clip(s1_aoi)

var visParams = {
  bands: l8bands,
  min: 0,
  max: 3000,
  gamma: 1.4,
};

// Display the results.
//Map.addLayer(pre_c,{min:-30,max:0},'pre')
//Map.addLayer(post_c,{min:-30,max:0},'post')
//Map.addLayer(s2_masked, {max: 4000}, 'Sentinel 2')
//Map.addLayer(l8, visParams, 'Landsat 8 SR');
//Map.addLayer(slope_masked);
//Map.addLayer(water_masked, {min:0,max:1, palette:['white','blue']}, 'possible inundated areas water masked')
Map.addLayer(slope_masked, {min:0,max:1, palette:['white','blue']}, 'possible inundated areas slope masked')
var ndsi = (post_c.subtract(pre_c)).divide(post_c.add(pre_c))
var f = ndsi.gte(0.15) //fixed value based on normalized sigma index
var NDSI_SMOOTH_RADIUS = 50 //meters
var filter = f.focal_median(NDSI_SMOOTH_RADIUS, 'circle', 'meters')
Map.addLayer(f, {palette:['white','blue']},'Raw NDSI')
Map.addLayer(ndsi, {min:-0.5, max:0.5, palette:['black', 'white', 'blue']}, 'NDSI Image')
Map.addLayer(filter, {palette:['white','blue']},'Filtered NDSI')

//Map.addLayer(occurrence, occurrenceVis, 'Occurrence');
//var popVis = pop.where(pop.gt(0), pop.log())
//  .visualize({palette: ['black','white']});
//Map.addLayer(popVis, {}, 'World Population (WorldPop');
Map.addLayer(ghs_bu,{}, 'Global Human Settlements (JRC)');
//Map.addLayer({
//  eeObject: water_mask,
//  visParams: VIS_WATER_MASK,
//  name: '20% occurrence water mask'
//});
//Map.addLayer(elevation.updateMask(lowland), {min: 0, max: 30, palette:['red','orange','yellow']}, 'floodplain');

// Bits 10 and 11 are clouds and cirrus, respectively.
var cloudBitMask = ee.Number(2).pow(10).int();
var cirrusBitMask = ee.Number(2).pow(11).int();

var qa = select.select('QA60');

//Map.addLayer(qa.bitwiseAnd(cloudBitMask).neq(0), {}, 's2 clouds');
//Map.addLayer(qa.bitwiseAnd(cirrusBitMask).neq(0), {}, 's2 cirrus');

function maskS2clouds(image) {
  var qa = image.select('QA60');
  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(
             qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask);
}

var cloudMasked = s2collection.filterBounds(s1_aoi).map(maskS2clouds);

var median = cloudMasked.median().clip(s1_aoi);
//print(median)
//Map.addLayer(median, {bands: ['B4', 'B3', 'B2'], max: 2000}, 'median');
''''''